# Significant words

https://moodle2.uni-leipzig.de/pluginfile.php/2889299/mod_resource/content/0/distant_readingII.pdf

In [4]:
import pandas as pd
#df = pd.read_parquet('sources/metadata.parquet', engine='pyarrow')
#df_large = pd.read_parquet('sources/metadata-large.parquet', engine='pyarrow')

exact_matches = pd.read_parquet("../results/artists_exact_match_large.parquet", engine='pyarrow')
assert "artists" in exact_matches.columns, f'artists is not in {exact_matches.columns}'
assert "num_artists" in exact_matches.columns, f'num_artists is not in {exact_matches.columns}'

print(f'exact match shapes {exact_matches.shape}')

# read excel_artist_names
import my_utils

excel_artist_names = my_utils.read_lines_as_list("../sources/excel_artists_copy_paste_name.txt")
hundred_artist_names = excel_artist_names[0:100]
ten_artist_names = excel_artist_names[0:10]


artist_mentions = pd.read_parquet('../results/artist_mentions.parquet', engine='pyarrow')

exact match shapes (1819808, 5)


In [2]:
import wordcloud
from collections import defaultdict
import re

def read_stopwords():
    # from https://gist.github.com/sebleier/554280
    return my_utils.read_lines_as_list("stopwords.txt")

def compute_corpus(df, col):
    # print(sw)
    corpus = []
    for i, row in df.iterrows():
        string = row[col]
        words = []
        for s in re.split("[, \-!?:]+", string):
            s=s.strip().lower()
            if len(s)>0:
                words.append(s)
        corpus.append(" ".join(words))
    return corpus
    

In [13]:
corpus = exact_matches.prompt.to_list()
print(len(corpus))
print(corpus[0])

1819808
beautiful porcelain ivory fair face woman biomechanical cyborg, close - up, sharp focus, studio light, iris van herpen haute couture headdress made of rhizomorphs, daisies, brackets, colorful corals, fractal mushrooms, puffballs, octane render, ultra sharp, 8 k 


In [5]:
stopwords = read_stopwords()

#corpus = compute_corpus(exact_matches, "prompt")
#print(len(corpus))
#print(corpus[0])

In [ ]:
#greg_df = my_utils.exact_match_dataframe(exact_matches, "greg rutkowski")
#freqs_greg = compute_freq(greg_df, "prompt")
#freqs_greg = remove(freqs_greg, ["greg", "rutkowski"])
#compute_wc(freqs_greg, "greg.png", top)
#print(f'finished')

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
#corpus = [
#    'This is the first document.',
#    'This document is the second document.',
#    'And this is the third one.',
#    'Is this the first document?',
#]
vectorizer = TfidfVectorizer(stop_words=stopwords)
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names_out()
print(f'{X.shape[0]} documents')
print(f'{X.shape[1]} words')
print(feature_names)
#print(X)
#print(f'Stopwords {vectorizer.get_stop_words()}')

1819808 documents
171528 words
['00' '000' '0000' ... '𝙁𝙧𝙞𝙚𝙙𝙧𝙞𝙘𝙝' '𝟖𝖐' '𝟭𝟴𝟬𝟬']


In [ ]:

#print(vectorizer.inverse_transform(X))

In [ ]:
# this dumb way takes ages!!!
#mean_tf_idf = {}
#for index, word in enumerate(feature_names):
    #print(word)
#    idf_sum=0
#    for doc in range(X.shape[0]):
#        #print(f'X[{doc}, {index}] is {X[doc, index]}')
#        idf_sum+=X[doc, index]
#    #print(f'{word} {idf_sum/X.shape[0]}')
#    mean_tf_idf[word] = idf_sum/X.shape[0]
#    
#    print(f'{index} {word} {mean_tf_idf[word]}')
#    break

In [ ]:
import pandas as pd

mean_tf_idfs = X.mean(axis=0)
print(mean_tf_idfs.shape)
mapping={}
print(mean_tf_idfs[0,0])
for index, fn in enumerate(feature_names):
    mapping[fn]=mean_tf_idfs[0,index]

df = pd.DataFrame.from_dict(mapping, orient="index", columns = ["tf_idf"])

df.sort_values("tf_idf", axis=0, ascending=False, inplace=True)
print(df.head(20))

# the average tf*idf is not computed over the non_zero values

When computing only over non_zero weights, we get total garbage (low frequency words).

In [26]:
# mean of non-zero
import numpy as np
print(X.shape)
sums = np.sum(X, axis=0)
print(sums)
print(sums.shape)
print(sums[0, 0])

nonzero_mean = sums

from collections import defaultdict
print(X.nonzero())
print(X.nonzero()[0].shape)
print(X.nonzero()[1].shape)
word_weight_indices = X.nonzero()[1]
word_occurences = defaultdict(lambda: 0)
for w in word_weight_indices:
    word_occurences[w]=word_occurences[w]+1

for index in range (nonzero_mean.shape[1]): #iterate over the elements
    amount_nonzero = word_occurences[index]
    #X.getcol(index).nonzero()[0].shape[0]
    if index==0:
        print(f'should be 62: {amount_nonzero}')
        assert amount_nonzero == 62
    tf_idf_sum = sums[0, index]
    nonzero_mean[0, index] = tf_idf_sum / amount_nonzero
    #if tf_idf_sum / amount_nonzero == 1:
    #    print(f'div is one for {amount_nonzero} mentions')


mean_tf_idfs = nonzero_mean
print(mean_tf_idfs.shape)
mapping={}
print(mean_tf_idfs[0,0])
for index, fn in enumerate(feature_names):
    mapping[fn]=[mean_tf_idfs[0,index], word_occurences[index]]

df = pd.DataFrame.from_dict(mapping, orient="index", columns = ["tf_idf", "query_mentions"])

df.sort_values("tf_idf", axis=0, ascending=False, inplace=True)
print(df.head())
print(df[df["query_mentions"]>100].head())

(1819808, 171528)
[[21.84440181 42.78653464  0.69930387 ...  0.38929793  0.14586499
   0.21821789]]
(1, 171528)
21.844401810916068
(array([      0,       0,       0, ..., 1819807, 1819807, 1819807],
      dtype=int32), array([156181, 125509, 108202, ..., 111591,  61803, 144632], dtype=int32))
(30767498,)
(30767498,)
should be 62: 62
(1, 171528)
0.3523290614663882
                 tf_idf  query_mentions
mauvey              1.0               1
solastalgia         1.0               2
gababooboobabba     1.0               1
gababooba           1.0               1
schnäbi             1.0               1
          tf_idf  query_mentions
dog     0.307528           11868
trump   0.306534           11084
car     0.277406           11355
emma    0.273945           10250
riding  0.267862           10896


In [11]:
from collections import defaultdict
print(X.nonzero())
print(X.nonzero()[0].shape)
print(X.nonzero()[1].shape)
word_weight_indices = X.nonzero()[1]
word_occurences = defaultdict(lambda: 0)
for w in word_weight_indices:
    word_occurences[w]=word_occurences[w]+1
print('finished')
print(word_occurences)
    
    
col0=X.getcol(0)
nz = col0.nonzero()
print(nz)
print(nz[0].shape)
print(nz[1].shape)
print(col0.shape)
print(col0)



inverse = vectorizer.inverse_transform(X)
print(inverse)
print(inverse.shape)

(array([0, 0, 1, 1, 2, 2, 3, 3], dtype=int32), array([0, 1, 3, 0, 2, 4, 0, 1], dtype=int32))
(8,)
(8,)
finished
defaultdict(<function <lambda> at 0x148039518790>, {0: 3, 1: 2, 3: 1, 2: 1, 4: 1})
(array([0, 1, 3], dtype=int32), array([0, 0, 0], dtype=int32))
(3,)
(3,)
(4, 1)
  (0, 0)	0.6292275146695526
  (1, 0)	0.78722297610404
  (3, 0)	0.6292275146695526
[array(['document', 'first'], dtype='<U8'), array(['second', 'document'], dtype='<U8'), array(['one', 'third'], dtype='<U8'), array(['document', 'first'], dtype='<U8')]


AttributeError: 'list' object has no attribute 'shape'

# Check if results are really different from frequency ranking

Yes they are different but not by much.

In [ ]:
from collections import defaultdict
import re

def read_stopwords():
    # from https://gist.github.com/sebleier/554280
    return my_utils.read_lines_as_list("stopwords.txt")

def compute_freq(df, col):
    sw = read_stopwords()
    # print(sw)
    freq = defaultdict(lambda: 0)
    for i, row in df.iterrows():
        string = row[col]
        for s in re.split("[, \-!?:]+", string):
            s=s.strip().lower()
            if s not in sw:
                if len(s)>0:
                    freq[s]=freq[s]+1
    return freq
    
def remove(freqs, words):
    
    result = defaultdict(lambda: 0)
    for key, value in freqs.items():
        #print(f'key {key}, value {value}')
        if key not in words:
            result[key]=value
    return result

freqs = compute_freq(exact_matches, "prompt")
freqs = remove(freqs, ["greg", "rutkowski"])
print(f'finished')

freqs_df = pd.DataFrame.from_dict(freqs, orient="index", columns = ["counts_plain"])
freqs_df.index.name = 'word_plain'

freqs_df.sort_values("counts_plain", axis=0, ascending=False, inplace=True)
#freqs_df['length']=freqs_df.word_plain.apply(lambda x: len(x))
freqs_df.reset_index(inplace=True)
freqs_sum = freqs_df.counts_plain.sum()
freqs_df['frequency_plain']=freqs_df.counts_plain.apply(lambda x: x / freqs_sum)

print(freqs_df.head(20))